In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [25]:
df1 = pd.read_csv("C://Users//bhask//Downloads//Customers.csv")
df2 = pd.read_csv("C://Users//bhask//Downloads//Products.csv")
df3 = pd.read_csv("C://Users//bhask//Downloads//Transactions.csv")

In [31]:
data = pd.merge(df3,df1, on="CustomerID", how="inner")
data = pd.merge(data, df2, on="ProductID", how="inner")

In [33]:
customer_summary = data.groupby("CustomerID").agg({
    "TotalValue": "sum",  # Total spending per customer
    "TransactionID": "count",  # Number of transactions per customer
    "ProductName": lambda x: x.mode()[0],  # Most purchased product
    "Category": lambda x: x.mode()[0],  # Most purchased category
}).reset_index()

In [35]:
customer_summary = customer_summary.merge(df1[["CustomerID", "Region"]], on="CustomerID", how="left")

In [37]:
# Handle categorical data Region and Category
customer_summary = pd.get_dummies(customer_summary, columns=["Region", "Category"])

In [39]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(customer_summary.drop(columns=["CustomerID", "ProductName"]))

In [41]:
# Compute Similarity Scores
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(scaled_features)

In [43]:
lookalike_map = {}
for idx, customer_id in enumerate(customer_summary["CustomerID"]):
    # Get similarity scores for the current customer
    scores = list(enumerate(similarity_matrix[idx]))
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    scores = [(customer_summary["CustomerID"].iloc[i], score) for i, score in scores if i != idx]
    # it takes the top 3 most similar customers
    lookalike_map[customer_id] = scores[:3]

In [45]:

lookalike_data = []
for customer_id, similar_customers in lookalike_map.items():
    lookalike_data.append({
        "CustomerID": customer_id,
        "Lookalikes": [{"CustomerID": sim_id, "Score": round(score, 2)} for sim_id, score in similar_customers]
    })
lookalike_df = pd.DataFrame(lookalike_data)
lookalike_df.to_csv("Lookalike.csv", index=False)
print("Lookalike recommendations generated and saved to Lookalike.csv!")

Lookalike recommendations generated and saved to Lookalike.csv!


In [47]:
lok=pd.read_csv("Lookalike.csv")
lok.head(20)

,CustomerID,Lookalikes
0,C0001,"[{'CustomerID': 'C0190', 'Score': 1.0}, {'Cust..."
1,C0002,"[{'CustomerID': 'C0088', 'Score': 0.99}, {'Cus..."
2,C0003,"[{'CustomerID': 'C0052', 'Score': 1.0}, {'Cust..."
3,C0004,"[{'CustomerID': 'C0155', 'Score': 0.99}, {'Cus..."
4,C0005,"[{'CustomerID': 'C0186', 'Score': 1.0}, {'Cust..."
5,C0006,"[{'CustomerID': 'C0168', 'Score': 0.99}, {'Cus..."
6,C0007,"[{'CustomerID': 'C0005', 'Score': 1.0}, {'Cust..."
7,C0008,"[{'CustomerID': 'C0065', 'Score': 0.91}, {'Cus..."
8,C0009,"[{'CustomerID': 'C0062', 'Score': 0.99}, {'Cus..."
9,C0010,"[{'CustomerID': 'C0062', 'Score': 0.99}, {'Cus..."
